In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import xarray as xr
from skdownscale.pointwise_models import AnalogRegression, PureAnalog

In [ ]:
# open a small dataset for training
training = xr.open_zarr("../data/downscale_test_data.zarr.zip", group="training")
training

In [ ]:
# open a small dataset of observations (targets)
targets = xr.open_zarr("../data/downscale_test_data.zarr.zip", group="targets")
targets

In [ ]:
# extract 1 point of training data for precipitation and temperature
X_temp = training.isel(point=0).to_dataframe()[["T2max"]] - 273.13
X_pcp = training.isel(point=0).to_dataframe()[["PREC_TOT"]] * 24
display(X_temp.head(), X_pcp.head())

In [ ]:
# extract 1 point of target data for precipitation and temperature
y_temp = targets.isel(point=0).to_dataframe()[["Tmax"]]
y_pcp = targets.isel(point=0).to_dataframe()[["Prec"]]
display(y_temp.head(), y_pcp.head())

In [ ]:
# Fit/predict using the PureAnalog class
for kind in ["best_analog", "sample_analogs", "weight_analogs", "mean_analogs"]:
    pure_analog = PureAnalog(kind=kind, n_analogs=10)
    pure_analog.fit(X_temp[:1000], y_temp[:1000])
    out = pure_analog.predict(X_temp[1000:])

    plt.plot(out[:300], label=kind)

# Fit/predict using the AnalogRegression class
analog_reg = AnalogRegression(n_analogs=100)
analog_reg.fit(X_temp[:1000], y_temp[:1000])
out = analog_reg.predict(X_temp[1000:])
plt.plot(out[:300], label="AnalogRegression")
plt.legend()